In [ ]:
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau
from keras.utils.np_utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator

# 1. Get Data

In [ ]:
def get_data(data_set, label_column='label'):
    # data_set:   pd DataFrame
    data_X = data_set.drop(columns=label_column)
    data_y = data_set[label_column]
    return data_X, data_y

In [ ]:
# read data
train_set = pd.read_csv('/kaggle/input/Kannada-MNIST/train.csv')
test_set = pd.read_csv('/kaggle/input/Kannada-MNIST/test.csv')
dig_set = pd.read_csv('/kaggle/input/Kannada-MNIST/Dig-MNIST.csv')

In [ ]:
X_train, y_train = get_data(train_set, label_column='label')
X_dig, y_dig = get_data(dig_set, label_column='label')

In [ ]:
X_train.shape, y_train.shape, X_dig.shape, y_dig.shape

In [ ]:
X_train = X_train.to_numpy().reshape(-1, 28, 28, 1)
y_train = y_train.to_numpy().reshape(-1, 1)
X_dig = X_dig.to_numpy().reshape(-1, 28, 28, 1)
y_dig = y_dig.to_numpy().reshape(-1, 1)

In [ ]:
test_set = test_set.drop(columns='id').to_numpy().reshape(-1, 28, 28, 1)

In [ ]:
y_train_cat = to_categorical(y_train)
y_dig_cat = to_categorical(y_dig)

In [ ]:
class_names = np.arange(10)

plt.figure(figsize=(10,10))
for i in range(25):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(X_train[i,:,:,0], cmap='gray')
    # y_train_cat is in one-hot format
    plt.xlabel(np.where(y_train_cat[i]==1)[0][0])
plt.show()

# 2. Image Generator

In [ ]:
d_width = 2
d_height = 2
zoom_range = [0.8,1.2]
brightness_range = [0.5,1.0]
fill_mode = 'nearest'
datagen = ImageDataGenerator(brightness_range=brightness_range, zoom_range=zoom_range,
                             width_shift_range=[-d_width, d_width],
                             height_shift_range=[-d_height, d_height], fill_mode=fill_mode )

In [ ]:

it = datagen.flow(X_train[0:10], batch_size=1)
plt.figure(figsize=(5, 5))
for i in range(9):
    plt.subplot(330 + 1+ i)
    batch = it.next()
    image = batch[0].astype('uint8')
    # print(image.shape)
    plt.imshow(image[:,:,0], cmap='gray')


# 3. Build Model

In [ ]:
# conv layers
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(layers.MaxPooling2D((2, 2)))
# add dropout
model.add(layers.Dropout(0.5))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Dropout(0.5))

# Fully connected layers
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

In [ ]:
# Adam optimizer
learning_rate = 0.001
beta_1 = 0.9
beta_2 = 0.999
optimizer = Adam(lr=learning_rate, beta_1=beta_1, beta_2=beta_2)

In [ ]:
model.compile(optimizer=optimizer,
              loss=['categorical_crossentropy'],
              metrics=['accuracy'])

In [ ]:
batch_size = 32
# reduce learning rate
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                              patience=10, min_lr=0.0001)
history = model.fit(datagen.flow(X_train, y_train_cat, batch_size=batch_size), callbacks=[reduce_lr],
          steps_per_epoch=len(X_train) / batch_size, epochs=10, validation_data=(X_dig, y_dig_cat))

In [ ]:
history.history

In [ ]:
plt.plot(history.history['acc'], label='accuracy')
plt.plot(history.history['val_acc'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(loc='lower right')

In [ ]:
start= time.time()
y_pred = model.predict_classes(test_set)
end = time.time()
test_time = (end - start) / 60
print(f'testing time: {test_time}')

In [ ]:
y_pred_pd = pd.DataFrame({'id': np.arange(len(y_pred)),
                          'label': y_pred})
y_pred_pd.to_csv('./y_pred.csv', sep=',', index=False)